<a href="https://colab.research.google.com/github/Areyouneema/Areyouneema/blob/main/week5_celebal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb

import warnings
warnings.filterwarnings("ignore")


In [5]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
print(train.shape, test.shape)

y = np.log1p(train['SalePrice'])  # log-transform target
train.drop(['SalePrice'], axis=1, inplace=True)
combined = pd.concat([train, test], sort=False)


(1460, 81) (1459, 80)


In [6]:
# Fill NA values
for col in combined.columns:
    if combined[col].dtype == "object":
        combined[col] = combined[col].fillna("None")
    else:
        combined[col] = combined[col].fillna(combined[col].median())

# One-hot encode
combined = pd.get_dummies(combined)

# Split back
X = combined[:len(y)]
X_test = combined[len(y):]


In [7]:
ridge = Ridge(alpha=10)
lasso = Lasso(alpha=0.001)

ridge.fit(X, y)
lasso.fit(X, y)

ridge_pred = np.expm1(ridge.predict(X_test))
lasso_pred = np.expm1(lasso.predict(X_test))


In [8]:
xgb_model = xgb.XGBRegressor(n_estimators=500, learning_rate=0.05)
xgb_model.fit(X, y)
xgb_pred = np.expm1(xgb_model.predict(X_test))


In [10]:
submission = pd.read_csv('sample_submission.csv')
submission['SalePrice'] = xgb_pred
submission.to_csv('submission.csv', index=False)


In [11]:
submission.head()

,Id,SalePrice
0,1461,127056.937500
1,1462,157512.437500
2,1463,188004.421875
3,1464,190418.593750
4,1465,187932.531250


In [12]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Id         1459 non-null   int64  
 1   SalePrice  1459 non-null   float32
dtypes: float32(1), int64(1)
memory usage: 17.2 KB
